In [3]:
import numpy as np
import helper_func
from sklearn import linear_model
import cPickle as pickle
import models
from sklearn.metrics.pairwise import cosine_similarity as cosdist
import edward as ed
from edward.models import Poisson,Gamma
import tensorflow as tf

In [ ]:
# PIK = "../models/bibtex_hpf_50.dat"
# with open(PIK, "rb") as f:
#     u,_,vy= pickle.load(f)
# f.close()
# dataset = 'multi_bibtex'
# x_train,y_train,x_test,y_test = helper_func.load_data(dataset)
# k = u.shape[1]
# w = np.zeros(shape=[x_train.shape[1],u.shape[1]])

In [9]:
sess = tf.InteractiveSession()
init = tf.global_variables_initializer()
init.run()
PIK = "../models/bibtex_hpf_50.dat"
with open(PIK, "rb") as f:
    [[a_s,bs],[_,_],[av2,bv2]]= pickle.load(f)
f.close()
dataset = 'multi_bibtex'
x_train,y_train,x_test,y_test = helper_func.load_data(dataset)

users = a_s.shape[0]
k = a_s.shape[1]
items2 = av2.shape[0]
q_theta = Gamma(a_s,bs)
theta_sample = q_theta.sample(30).eval()
u = np.zeros(shape=[users,k])
for i in range(0,30):
    u += theta_sample[i]
u /= 30

q_beta2 = Gamma(np.transpose(av2),np.transpose(bv2))
beta2_sample = q_beta2.sample(30).eval()
vy = np.zeros(shape=[k,items2])
for i in range(0,30):
    vy += beta2_sample[i]
vy /= 30

# k = u.shape[1]
w = np.zeros(shape=[x_train.shape[1],u.shape[1]])

In [10]:
temp = np.matmul(u,vy)
result = 0.0
for i in range(0,temp.shape[0]):
    result += helper_func.patk(temp[i],y_train[i],1)
result /= temp.shape[0]
print result

0.465983606557


In [14]:
for i in range(0,k):
    print i
    reg = linear_model.Ridge(alpha = 0.2)
    reg.fit(x_train,u[:,i])
    w[:,i] = reg.coef_
u_hat = np.matmul(x_train,w)
u_test = np.matmul(x_test,w)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [15]:
nb = 150
result = 0.0
for i in range(0,u_test.shape[0]):
    dist = cosdist(np.reshape(u_test[i],newshape=[1,-1]),u_hat)
    indices = np.argsort(dist)[:,:nb]
    predicted = np.zeros(shape=y_train.shape[1],dtype=np.float64)
    for j in range(0,nb):
        predicted += y_train[indices[0,j]]
    predicted /= nb
    result += helper_func.patk(predicted,y_test[i],1)
result /= u_test.shape[0]
print result

0.00159045725646
